In [ ]:
import numpy as np

In [ ]:
kc1 = 300; # production of p53 [nM/h]
kc2 = 1 # degradation of p53 by mdm2 [1/h]
kc3 = 6; # degradation of p53 by mdm2 [nM]
kc4 = 0.03; # production of mdm2 - mRNA [1/(nM*h)]
kc5 = 0.5 # degradation of mdm2 - mRNA [1/h]
kc6 = 1.4 # production of mdm2 [1/h]
kc7 = 0.6 # degradation of mdm2 [1/h] 0.2
kc8 = 0.01 # binding of Mdm2 and nutlin [1/h]
kc9 = 1 # unbinding of Mdm2 and nutlin [1/h]

In [ ]:
def f_p53(t,x,kc1,kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc9):
    m,Mfree,p,Mbound,n = x
    dpdt = kc1-kc2*Mfree*p/(kc3+p) #p53
    dmdt = kc4*p**2-kc5*m # mdm2 mRNA
    dMfreedt = kc6*m-kc8*n*Mfree-kc7*Mfree + kc9*Mbound # free Mdm2
    dMbounddt = -kc7*Mbound+kc8*n*Mfree - kc9*Mbound # bound Mdm2 (complex with nutlin)
    dndt=-kc8*n*Mfree + kc9*Mbound # nutlin
    return np.array([dmdt,dMfreedt,dpdt,dMbounddt, dndt])

def odeRungeKutta4_p53(f,x0,dt,simT, tstart_nut,tend_nut, t_ON, t_OFF, nut_const, args = ()):

    '''tstart_nut : time of start of nutlin stimulation
    tend_nut : time of end of nutlin stimulation
    t_ON : time interval nutlin is left in the system
    t_OFF : time interval nutlin is washed off
    if t_ON = t_OFF = T_ext / 2 the system has equal times on and off of nutlin'''

    t = np.arange(0, simT, dt)
    dose=np.zeros(len(t))
    ntot=np.zeros(len(t))
    x = np.zeros((len(t),len(x0)))
    x[0,:] = x0
    dt2 = dt/2.0
    twash_nut = tstart_nut+t_ON
    while twash_nut <= tend_nut: #Define nutlin pulses
        # int(tstart_nut/dt) gives the index in the placeholder dose array which we are currently at after running for current no. of dt's
        dose[int(tstart_nut/dt):int(tstart_nut/dt) + 1] = nut_const # give pulse of nutlin for 1 dt
        ntot[int(tstart_nut/dt):int(twash_nut/dt)] = 1 # set nutlin to 0 outside each interval tstart-twash
        tstart_nut += t_ON+t_OFF # Add one period to each array
        twash_nut += t_ON+t_OFF
    for n in range(0,len(t)-1):
        K1 = dt*f(t[n],x[n,:], *args)
        K2 = dt*f(t[n] + dt2, x[n,:] + 0.5*K1, *args)
        K3 = dt*f(t[n] + dt2, x[n,:] + 0.5*K2, *args)
        K4 = dt*f(t[n] + dt, x[n,:] + K3, *args)
        x[n+1, :] = x[n,:] + (1/6.0)*(K1 + 2*K2 + 2*K3 + K4)
        x[n+1, -1] = ntot[n] * (x[n+1, -1] + dose[n]) # Set last element of nutlin level array to be equal to current value + dose * either 1 or 0. It is only 1 where the pulses have been given.
        for i in range(len(x0)):
            x[n+1,i] = max(0,x[n+1,i]) # set min value to 0
    return x

In [ ]:
x = odeRungeKutta4_p53(f_p53, init_cond, dt, simT, tstart_nut,tend_nut,
    t_ON, t_OFF, nut_const, args = (kc1,kc2,kc3,kc4,kc5,kc6,kc7, kc8, kc9))